## Set up

In [1]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from gensim.models import Phrases
from gensim.models.phrases import Phraser 
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from pprint import pprint

In [2]:
data = pd.read_csv("data/cleaned_data2.csv")

/var/folders/5q/yq3hk8g1793ckqmn2n3xpr6c0000gn/T/ipykernel_18704/2170368664.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/cleaned_data2.csv")


## Preprocessing

Tokenizer

In [3]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
data["tokenized"] = data["cleaned_text"].apply(lambda x: tokenizer.tokenize(x))

Stop words

In [4]:
#Custom defined list
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "wasnt", "yeah", "okay", "lol"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "op"]

topic_specific = ["self", "improvement", "selfimprovement", "rselfimprovement"]

stop_words = english + time + reddit + topic_specific

In [5]:
#Remove stopwords
def remove_stopwords_numbers(tokens):
    """Removes stopwords and tokens that contain any digits"""
    return [token for token in tokens if token not in stop_words and not re.search(r'\d', token)]

# Apply function to column
data["filtered_tokens"] = data["tokenized"].apply(remove_stopwords_numbers)

In [6]:
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    """Lemmatizes a list of tokens"""
    return [lemmatizer.lemmatize(token) for token in tokens]

# Apply function to column
data["lemmatized"] = data["filtered_tokens"].apply(lemmatize_tokens)

In [7]:
#https://www.h2kinfosys.com/blog/part-of-speech-tagging-chunking-with-nltk/
def get_wordnet_pos(tag):
    """Convert POS tag to a format recognized by WordNetLemmatizer"""
    if tag.startswith('J'): #NLTK tags adjectives as JJ (normal), 
                            #JJR (comparative) and JJS (superlative)
        return wordnet.ADJ  #and then it can be mapped to the wordnet database
                              #to get the lemma
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_tokens(tokens):
    """Lemmatizes tokens with POS tagging"""
    tagged_tokens = pos_tag(tokens)  
    return [lemmatizer.lemmatize(word, pos) for word, tag in tagged_tokens
            if (pos := get_wordnet_pos(tag))]

data["lemmatized"] = data["filtered_tokens"].apply(lemmatize_tokens)

Bigrams

In [8]:
# Train bigram model on the lemmatized column
bigram = Phrases(data["lemmatized"], min_count=20)
bigram_phraser = Phraser(bigram)  # Optimizes since it is a lighter-weight
                                   #version of Phrases

def add_bigrams(tokens):
    """Adds bigrams to a list of tokens if they appear frequently enough."""
    bigram_tokens = bigram_phraser[tokens]
    return tokens + [token for token in bigram_tokens if '_' in token]

data["ngrams"] = data["lemmatized"].apply(add_bigrams)

Filtering of too rare of too common words

In [9]:
# Convert df column to a list of lists for later steps
docs = data["ngrams"].tolist() 

# Create a dictionary from the tokenized documents
dictionary = Dictionary(docs)

# Filter out words that appear in fewer than 20 documents or 
#  more than 80% of documents, and keep the 1500 most frequent
dictionary.filter_extremes(no_below=20, no_above=0.8, keep_n=1500)


Vectorize

In [10]:
# Create bag of words representation
corpus = [dictionary.doc2bow(doc) for doc in docs]

## Training

### Test 1

In [11]:
#Set hyperparameters
num_topics = 8
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model1 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model1.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model1, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model1.save("lda_models/model1") #CHANGE

[([(0.023322126, 'even'),
   (0.021146834, 'now'),
   (0.013151883, 'never'),
   (0.013144989, 'bad'),
   (0.012929399, 'right'),
   (0.012910244, 'well'),
   (0.012796486, 'hard'),
   (0.0117129255, 'always'),
   (0.011606118, 'back'),
   (0.010873882, 'best'),
   (0.010700444, 'friend'),
   (0.009930863, 'help'),
   (0.009189448, 'first'),
   (0.0086915195, 'problem'),
   (0.0086060995, 'actually'),
   (0.008497644, 'year'),
   (0.008123384, 'everything'),
   (0.0078123384, 'sure'),
   (0.0077694952, 'try'),
   (0.0077583473, 'care')],
  -1.749075479801176),
 ([(0.030667972, 'start'),
   (0.029763892, 'goal'),
   (0.021765677, 'book'),
   (0.019902375, 'step'),
   (0.019636743, 'small'),
   (0.01939953, 'new'),
   (0.0177166, 'habit'),
   (0.017436445, 'use'),
   (0.01579925, 'brain'),
   (0.014720536, 'easy'),
   (0.013386877, 'focus'),
   (0.011758619, 'first'),
   (0.01072858, 'help'),
   (0.010627422, 'great'),
   (0.00997251, 'practice'),
   (0.008777292, 'free'),
   (0.00874485

## Test 2

In [12]:
#Set hyperparameters
num_topics = 10
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model2 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model2.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model2, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model2.save("lda_models/model2") #CHANGE

[([(0.023901934, 'even'),
   (0.021488396, 'now'),
   (0.013478846, 'never'),
   (0.013472212, 'bad'),
   (0.013407485, 'well'),
   (0.013266396, 'right'),
   (0.013114614, 'hard'),
   (0.012028285, 'always'),
   (0.011894259, 'back'),
   (0.011862702, 'best'),
   (0.011026301, 'help'),
   (0.01089096, 'first'),
   (0.010394852, 'try'),
   (0.009326888, 'problem'),
   (0.009073895, 'need'),
   (0.008905527, 'actually'),
   (0.008397691, 'everything'),
   (0.008214058, 'sure'),
   (0.008158584, 'look'),
   (0.00795312, 'care')],
  -1.8185322607572232),
 ([(0.050405886, 'start'),
   (0.047568902, 'goal'),
   (0.031808157, 'step'),
   (0.03138362, 'small'),
   (0.028314807, 'habit'),
   (0.024903404, 'brain'),
   (0.023712259, 'focus'),
   (0.018747238, 'easy'),
   (0.014824198, 'term'),
   (0.013500659, 'plan'),
   (0.012382608, 'use'),
   (0.011828485, 'motivation'),
   (0.011705776, 'practice'),
   (0.01160463, 'meditation'),
   (0.011418666, 'progress'),
   (0.011343988, 'build'),
   

## Test 3

In [13]:
#Set hyperparameters
num_topics = 12
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model3 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model3.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model3, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model3.save("lda_models/model3") #CHANGE TWICEEE

[([(0.02475291, 'even'),
   (0.022444153, 'now'),
   (0.013958728, 'never'),
   (0.013951843, 'bad'),
   (0.013884829, 'well'),
   (0.013738706, 'right'),
   (0.013581524, 'hard'),
   (0.012728771, 'best'),
   (0.012456515, 'always'),
   (0.012318128, 'back'),
   (0.010892975, 'first'),
   (0.010764928, 'try'),
   (0.010145202, 'year'),
   (0.009668328, 'help'),
   (0.009222178, 'actually'),
   (0.009136276, 'problem'),
   (0.008696667, 'everything'),
   (0.008647206, 'need'),
   (0.008506493, 'sure'),
   (0.008402017, 'long')],
  -1.7184996175427674),
 ([(0.05510784, 'start'),
   (0.04868412, 'goal'),
   (0.035601564, 'book'),
   (0.032553896, 'step'),
   (0.032119386, 'small'),
   (0.02897859, 'habit'),
   (0.02726805, 'new'),
   (0.017640864, 'easy'),
   (0.016057625, 'use'),
   (0.015554126, 'term'),
   (0.014303773, 'read'),
   (0.01408229, 'plan'),
   (0.013528973, 'first'),
   (0.012105746, 'motivation'),
   (0.011686352, 'progress'),
   (0.011609969, 'build'),
   (0.011371011, 

## Test 4

In [14]:
#Set hyperparameters
num_topics = 14
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model4 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model4.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model4, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model4.save("lda_models/model4") #CHANGE TWICEEE

[([(0.015662765, 'best'),
   (0.015480469, 'bad'),
   (0.01499854, 'help'),
   (0.013931213, 'others'),
   (0.012714986, 'change'),
   (0.012363899, 'need'),
   (0.012163537, 'try'),
   (0.010914411, 'problem'),
   (0.010683092, 'well'),
   (0.010394388, 'actually'),
   (0.010112692, 'different'),
   (0.009872003, 'look'),
   (0.0096762385, 'always'),
   (0.009434592, 'sure'),
   (0.009383129, 'relationship'),
   (0.009306818, 'care'),
   (0.009299084, 'everyone'),
   (0.009008385, 'great'),
   (0.008806591, 'enough'),
   (0.008780333, 'point')],
  -1.9414695505118886),
 ([(0.048904493, 'now'),
   (0.047341634, 'even'),
   (0.03041479, 'never'),
   (0.029935831, 'right'),
   (0.029625326, 'first'),
   (0.029592322, 'hard'),
   (0.027784623, 'year'),
   (0.026840499, 'back'),
   (0.021726992, 'long'),
   (0.018609138, 'everything'),
   (0.015506672, 'little'),
   (0.013434328, 'big'),
   (0.013339688, 'away'),
   (0.013124464, 'time'),
   (0.012860087, 'ever'),
   (0.012417734, 'place')

## Test 5

In [15]:
#Set hyperparameters
num_topics = 8
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 0.1
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model5 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model5.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model5, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model5.save("lda_models/model5") #CHANGE TWICEEE

[([(0.033151537, 'job'),
   (0.023535356, 'money'),
   (0.023313778, 'year'),
   (0.021253522, 'now'),
   (0.017094431, 'school'),
   (0.012321521, 'old'),
   (0.011196199, 'even'),
   (0.0109509425, 'age'),
   (0.010071777, 'never'),
   (0.0095861545, 'kid'),
   (0.00926911, 'college'),
   (0.009106586, 'young'),
   (0.008888613, 'parent'),
   (0.008534545, 'back'),
   (0.008362082, 'high'),
   (0.008081797, 'family'),
   (0.0077813664, 'career'),
   (0.007151115, 'live'),
   (0.006927939, 'new'),
   (0.006874033, 'hard')],
  -1.9490944986206644),
 ([(0.019988524, 'now'),
   (0.0144819, 'help'),
   (0.01392344, 'even'),
   (0.012623926, 'bad'),
   (0.012129045, 'right'),
   (0.010393977, 'therapy'),
   (0.009661464, 'back'),
   (0.009628363, 'hard'),
   (0.009345396, 'well'),
   (0.009221945, 'mental'),
   (0.009081313, 'year'),
   (0.008785201, 'problem'),
   (0.0086882, 'feeling'),
   (0.008522278, 'relationship'),
   (0.008473866, 'never'),
   (0.008399801, 'issue'),
   (0.00825889

## Test 6

In [16]:
#Set hyperparameters
num_topics = 10
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 0.1
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model6 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model6.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model6, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model6.save("lda_models/model6") #CHANGE TWICEEE

[([(0.026990864, 'now'),
   (0.018591389, 'right'),
   (0.018399194, 'even'),
   (0.01725437, 'bad'),
   (0.012818934, 'never'),
   (0.012811984, 'back'),
   (0.011543322, 'hard'),
   (0.011334019, 'well'),
   (0.009942058, 'always'),
   (0.008867158, 'everything'),
   (0.008169842, 'change'),
   (0.0081667295, 'feeling'),
   (0.007200153, 'try'),
   (0.0070645977, 'right_now'),
   (0.0069945436, 'problem'),
   (0.0069034356, 'actually'),
   (0.006844626, 'best'),
   (0.006747732, 'year'),
   (0.006634837, 'sometimes'),
   (0.006448188, 'wrong')],
  -1.8460157121593002),
 ([(0.034252744, 'job'),
   (0.026403537, 'year'),
   (0.024317116, 'money'),
   (0.022270253, 'now'),
   (0.017662246, 'school'),
   (0.01363739, 'old'),
   (0.011906845, 'kid'),
   (0.011794237, 'even'),
   (0.011769575, 'age'),
   (0.011760355, 'parent'),
   (0.010942379, 'never'),
   (0.010692018, 'young'),
   (0.0098495055, 'family'),
   (0.009771607, 'back'),
   (0.009576985, 'college'),
   (0.008386108, 'high'),

## Test 7

In [17]:
#Set hyperparameters
num_topics = 14
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 0.1
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model7 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model7.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model7, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model7.save("lda_models/model7") #CHANGE TWICEEE

[([(0.0483676, 'now'),
   (0.034893326, 'right'),
   (0.022688296, 'even'),
   (0.015688933, 'back'),
   (0.015580429, 'hard'),
   (0.015511696, 'year'),
   (0.015485671, 'well'),
   (0.013632049, 'bad'),
   (0.013439983, 'best'),
   (0.013182737, 'right_now'),
   (0.011964829, 'never'),
   (0.011540899, 'always'),
   (0.01115677, 'bit'),
   (0.010462063, 'little'),
   (0.010374455, 'first'),
   (0.00996753, 'advice'),
   (0.009715144, 'try'),
   (0.009462831, 'actually'),
   (0.009011112, 'situation'),
   (0.008676303, 'everything')],
  -1.8186389410214834),
 ([(0.019263934, 'happy'),
   (0.018418165, 'love'),
   (0.018214244, 'never'),
   (0.016562913, 'relationship'),
   (0.016022842, 'family'),
   (0.014645117, 'now'),
   (0.013831138, 'friend'),
   (0.0128262825, 'even'),
   (0.0123596415, 'parent'),
   (0.011611842, 'care'),
   (0.01148698, 'child'),
   (0.01106954, 'man'),
   (0.01053636, 'live'),
   (0.010299108, 'shit'),
   (0.009679815, 'year'),
   (0.009574915, 'always'),
  

## Test 8

In [18]:
#Set hyperparameters
num_topics = 8
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 0.1

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model8 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model8.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model8, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model8.save("lda_models/model8") #CHANGE TWICEEE

[([(0.023491764, 'even'),
   (0.021300638, 'now'),
   (0.013247545, 'never'),
   (0.01324024, 'bad'),
   (0.013017959, 'right'),
   (0.012970394, 'well'),
   (0.012889559, 'hard'),
   (0.01179104, 'always'),
   (0.011690534, 'back'),
   (0.0108550675, 'friend'),
   (0.010398024, 'best'),
   (0.010008525, 'help'),
   (0.009238186, 'first'),
   (0.008778553, 'problem'),
   (0.008664443, 'actually'),
   (0.008618961, 'year'),
   (0.008171591, 'everything'),
   (0.007871212, 'sure'),
   (0.007813372, 'care'),
   (0.0077912374, 'try')],
  -1.7472625279798888),
 ([(0.03087114, 'start'),
   (0.029760417, 'goal'),
   (0.02176313, 'book'),
   (0.019900044, 'step'),
   (0.019699503, 'new'),
   (0.019634442, 'small'),
   (0.01771452, 'habit'),
   (0.017434401, 'use'),
   (0.015797397, 'brain'),
   (0.014898939, 'easy'),
   (0.013325246, 'focus'),
   (0.011815577, 'first'),
   (0.0106797675, 'help'),
   (0.010677372, 'great'),
   (0.010023587, 'practice'),
   (0.008897365, 'free'),
   (0.008743828

## Test 9 

In [19]:
#Set hyperparameters
num_topics = 14
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 0.1

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model9 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model9.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model9, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model9.save("lda_models/model9") #CHANGE TWICEEE

[([(0.048476476, 'even'),
   (0.04721232, 'now'),
   (0.029362831, 'never'),
   (0.028900005, 'right'),
   (0.028568529, 'hard'),
   (0.028513331, 'first'),
   (0.026853278, 'year'),
   (0.025911778, 'back'),
   (0.02097521, 'long'),
   (0.018129079, 'everything'),
   (0.014970124, 'little'),
   (0.014491783, 'always'),
   (0.012878121, 'away'),
   (0.012775032, 'big'),
   (0.012670348, 'time'),
   (0.012415114, 'ever'),
   (0.011825555, 'place'),
   (0.011242125, 'well'),
   (0.011233587, 'happy'),
   (0.010789325, 'bit')],
  -1.8909044992097919),
 ([(0.016154254, 'best'),
   (0.01555083, 'help'),
   (0.01524502, 'bad'),
   (0.01454248, 'others'),
   (0.013282796, 'change'),
   (0.012790227, 'need'),
   (0.012583588, 'try'),
   (0.0112907635, 'problem'),
   (0.01067768, 'actually'),
   (0.010461398, 'different'),
   (0.010212412, 'look'),
   (0.009983319, 'well'),
   (0.009636928, 'sure'),
   (0.009627737, 'care'),
   (0.009619736, 'everyone'),
   (0.009550385, 'great'),
   (0.0090421

## Test 10

In [20]:
#Set hyperparameters
num_topics = 10
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 0.1

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model10 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model10.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model10, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model10.save("lda_models/model10") #CHANGE TWICEEE

[([(0.023901934, 'even'),
   (0.021488396, 'now'),
   (0.013478846, 'never'),
   (0.013472212, 'bad'),
   (0.013407485, 'well'),
   (0.013266396, 'right'),
   (0.013114614, 'hard'),
   (0.012028285, 'always'),
   (0.011894259, 'back'),
   (0.011862702, 'best'),
   (0.011026301, 'help'),
   (0.01089096, 'first'),
   (0.010394852, 'try'),
   (0.009326888, 'problem'),
   (0.009073895, 'need'),
   (0.008905527, 'actually'),
   (0.008397691, 'everything'),
   (0.008214058, 'sure'),
   (0.008158584, 'look'),
   (0.00795312, 'care')],
  -1.8185322607572232),
 ([(0.050405886, 'start'),
   (0.047568902, 'goal'),
   (0.031808157, 'step'),
   (0.03138362, 'small'),
   (0.028314807, 'habit'),
   (0.024903404, 'brain'),
   (0.023712259, 'focus'),
   (0.018747238, 'easy'),
   (0.014824198, 'term'),
   (0.013500659, 'plan'),
   (0.012382608, 'use'),
   (0.011828485, 'motivation'),
   (0.011705776, 'practice'),
   (0.01160463, 'meditation'),
   (0.011418666, 'progress'),
   (0.011343988, 'build'),
   

## Test 11

In [21]:
#Set hyperparameters
num_topics = 12
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 0.1

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model11 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model11.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model11, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model11.save("lda_models/model11") #CHANGE TWICEEE

[([(0.024765339, 'even'),
   (0.022455433, 'now'),
   (0.013965743, 'never'),
   (0.013958856, 'bad'),
   (0.013890443, 'well'),
   (0.013745609, 'right'),
   (0.013588345, 'hard'),
   (0.012462775, 'always'),
   (0.012324317, 'back'),
   (0.012216835, 'best'),
   (0.0111214025, 'first'),
   (0.010770338, 'try'),
   (0.010139172, 'year'),
   (0.009627435, 'help'),
   (0.00922634, 'actually'),
   (0.009106984, 'problem'),
   (0.008701038, 'everything'),
   (0.008538338, 'need'),
   (0.0085107675, 'sure'),
   (0.008480964, 'long')],
  -1.7193027146486488),
 ([(0.021167617, 'others'),
   (0.014741483, 'change'),
   (0.014439111, 'experience'),
   (0.013979097, 'world'),
   (0.012976657, 'mind'),
   (0.012929394, 'different'),
   (0.009699207, 'feeling'),
   (0.0095436815, 'learn'),
   (0.00946373, 'important'),
   (0.007783708, 'focus'),
   (0.0077611334, 'human'),
   (0.0077571706, 'situation'),
   (0.007729595, 'positive'),
   (0.007510808, 'often'),
   (0.007455183, 'control'),
   (0.0